In [1]:
%load_ext autoreload
%autoreload 2

import sys; 
sys.path.insert(0, '../../esinet')
sys.path.insert(0, '../')

import numpy as np
from copy import deepcopy
from scipy.sparse.csgraph import laplacian
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
from invert.cmaps import parula
pp = dict(surface='white', hemi='both')

In [78]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
leadfield -= leadfield.mean(axis=0)
n_chans, n_dipoles = leadfield.shape
dist = cdist(pos, pos)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.7s remaining:    2.7s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.9s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished


In [110]:
# settings = dict(number_of_sources=1, extents=40, duration_of_trial=0.01, target_snr=99999999999)
settings = dict(number_of_sources=3, extents=(1, 2), duration_of_trial=0.1, target_snr=1e99)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()
y = evoked.data
x = stc.data
y -= y.mean(axis=0)

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 16.15it/s]

Using control points [0.00000000e+00 0.00000000e+00 3.68943151e-08]


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Linearly Constrained minimum variance Beamformer

In [111]:
alpha = 0.00 * np.linalg.svd(leadfield)[1].max()
leadfield -= leadfield.mean(axis=0)
I = np.identity(n_chans)
C_inv = np.linalg.inv(y@y.T + alpha * I)
W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    W.append(w)
W = np.stack(W, axis=1)[0].T
W = W / np.linalg.norm(W, axis=0)
inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat / x_hat.max()
stc_.plot(**pp, brain_kwargs=dict(title="LCMV"))

Using control points [4.98244354e-07 5.77181464e-07 2.07953722e-01]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Weight-normalized minimum variance Beamformer

In [123]:
alpha = 0.0001 * np.linalg.svd(leadfield)[1].max()

I = np.identity(n_chans)
C_inv = np.linalg.inv(y@y.T + alpha * I)
C_inv_2 = np.linalg.inv(C_inv)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    # w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    w = (C_inv @ l) / np.sqrt(l.T @ C_inv_2 @ l)
    W.append(w)
W = np.stack(W, axis=1)[:, :, 0]
# W = W / np.linalg.norm(W, axis=0)
inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat
stc_.plot(**pp, brain_kwargs=dict(title="WNMV"))

Using control points [3.32170934e-05 3.61216712e-05 5.07131357e-05]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [3.94027262e-05 4.14323823e-05 4.50397172e-05]
Using control points [3.94027262e-05 4.14323823e-05 4.50397172e-05]


# Standardized minimum variance Beamformer

In [126]:
alpha = 0.00001 #* np.linalg.svd(leadfield)[1].max()

I = np.identity(n_chans)
C_inv = np.linalg.inv(y@y.T + alpha * I)
C_inv_2 = np.linalg.inv(C_inv)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    # w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    w = (C_inv @ l) / np.sqrt(l.T @ C_inv @ l)
    W.append(w)
W = np.stack(W, axis=1)[:, :, 0]
# W = W / np.linalg.norm(W, axis=0)
inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat
stc_.plot(**pp, brain_kwargs=dict(title="SMV"))

Using control points [0.00895977 0.00974578 0.01369014]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [0.01063278 0.01118369 0.01216406]
Using control points [0.00011244 0.00011823 0.00012853]


# Higher-Order minimum variance Beamformer

In [127]:
alpha = 0.00 * np.linalg.svd(leadfield)[1].max()
n = 3
I = np.identity(n_chans)
C_inv = np.linalg.inv(y@y.T + alpha * I)
C_inv_n = deepcopy(C_inv)

for _ in range(n-1):
    C_inv_n = np.linalg.inv(C_inv_n)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    # w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    w = (C_inv_n @ l) / (l.T @ C_inv_n @ l)
    W.append(w)
W = np.stack(W, axis=1)[:, :, 0]
W = W / np.linalg.norm(W, axis=0)
inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat
stc_.plot(**pp, brain_kwargs=dict(title="HOC-MV"))

Using control points [1.93460237e-18 2.26793429e-18 1.61098280e-12]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [2.38517523e-20 2.81807238e-20 3.16121197e-13]
Using control points [0.00000000e+00 0.00000000e+00 3.56600339e-08]


# Eigenspace-based minimum variance Beamformer

In [128]:
from invert.util import find_corner
alpha = 0.00 * np.linalg.svd(leadfield)[1].max()

I = np.identity(n_chans)
C = y@y.T

U, s, _ = np.linalg.svd(C)
j = find_corner(np.arange(len(s)), s)

Us = U[:, :j]
Un = U[:, j:]


C_inv = np.linalg.inv(C + alpha * I)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    # w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    w_mv = (C_inv @ l) / (l.T @ C_inv @ l)
    w_esmv = Us @ Us.T @ w_mv
    W.append(w_esmv)

W = np.stack(W, axis=1)[:, :, 0]
# W = W / np.linalg.norm(W, axis=0)

inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat / x_hat.max()
stc_.plot(**pp, brain_kwargs=dict(title="EIGMV"))

Using control points [1.10495282e-14 1.89606539e-14 4.31939588e-01]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [1.96416299e-14 3.34455927e-14 5.47908045e-01]


# Multiple Constrained minimum variance Beamformer

In [131]:
L = deepcopy(leadfield)
L -= L.mean(axis=0)
# L /= np.linalg.norm(leadfield, axis=0)
alpha = 0.00 * np.linalg.svd(L)[1].max()

I = np.identity(n_chans)
C_inv = np.linalg.inv(y@y.T + alpha * I)
W = C_inv @ L @ np.linalg.inv(L.T @ C_inv @ L)
# W = W / np.linalg.norm(W, axis=0)

inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat / x_hat.max()
stc_.plot(**pp, brain_kwargs=dict(title="MCMV"))

Using control points [0.06348538 0.09014199 0.6293157 ]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Multiple Constrained Eigenspace MV Beamformer

In [ ]:
W = C_inv @ L @ np.linalg.inv(L.T @ C_inv @ L)

In [135]:
from invert.util import find_corner
alpha = 0.00 * np.linalg.svd(leadfield)[1].max()

I = np.identity(n_chans)
C = y@y.T

U, s, _ = np.linalg.svd(C)
j = find_corner(np.arange(len(s)), s)

Us = U[:, :j]
Un = U[:, j:]


C_inv = np.linalg.inv(C + alpha * I)

W = []
for i in range(n_dipoles):
    l = leadfield[:, i][:, np.newaxis]
    # w = np.linalg.inv(l.T @ C_inv @ l ) * l.T @ C_inv
    w_mv = C_inv @ l *(1 / (l.T @ C_inv @ l))
    w_esmv = Us @ Us.T @ w_mv
    W.append(w_esmv)

W = np.stack(W, axis=1)[:, :, 0]
# W = W / np.linalg.norm(W, axis=0)

inverse_operator = W.T
x_hat = inverse_operator@y
stc_ = stc.copy()
stc_.data = x_hat / x_hat.max()
stc_.plot(**pp, brain_kwargs=dict(title="EIG-MCMV"))

Using control points [1.09915358e-14 1.89182569e-14 4.31939590e-01]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [1.98206652e-14 3.32850217e-14 5.47908046e-01]
Using control points [0.38290875 0.44257064 1.08843471]
Using control points [0.38290875 0.44257064 1.08843471]
